In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

#(1) 加载数据
os.chdir('D:\\工作\\数据分析学习\\数据分析师（网易）\\【代码+软件+课后答案】课程资料\\【非常重要】Python数据分析师微专业_课程资料\\CLASSDATA_ch06数据挖掘实战')
df = pd.read_excel('上海餐饮数据.xlsx',sheetname = 0,header = 0)
print('加载数据%i 条' %len(df))


加载数据96398 条


In [38]:
# 查看数据范围
# print(df[df['口味'] == np.nan])
# print(df[df['环境'] == np.nan])
# print(df['服务'].max())
# print(df['人均消费'].max())

Empty DataFrame
Columns: [类别, 行政区, 点评数, 口味, 环境, 服务, 人均消费, 城市, Lng, Lat]
Index: []
Empty DataFrame
Columns: [类别, 行政区, 点评数, 口味, 环境, 服务, 人均消费, 城市, Lng, Lat]
Index: []


In [214]:
# (2) 数据清洗 + 添加字段
# 删除空白值，取口味人均消费均大于0的值，增加字段
data = df[['类别','口味','环境','服务','人均消费']]
data.dropna(inplace=True)
data = data[(data['口味']>0)&(data['人均消费']>0)]
data['性价比'] = (df['口味'] + df['环境'] + df['服务'])/df['人均消费']
print('处理数据%i条' %len(data[(data['口味']<=0)|(data['人均消费']<=0)]))

处理数据0条


In [215]:
s = data['口味'].describe()
s
print(s['25%'])

7.1


In [216]:
# （3）处理异常值
def f2(data,col):
    s = data[col].describe()     # 要是没有这一步的话，新版会报错（摊手）
    q1 = s['25%']
    q3 = s['75%']
    iqr = q3 - q1
    mi = q1 - 1.5 * iqr
    ma = q3 + 1.5 * iqr
#     data_e = data[(data[col] >= mi)&(data[col] <= ma)][['类别','服务']]
    return data[(data[col] >= mi)&(data[col] <= ma)][['类别','服务']]

# 做一次异常值清洗过后，其实还会有异常值，。。。一直都有异常值的

In [217]:
data_kw = f2(data, '口味')
data_rj = f2(data,'人均消费')
data_xjb = f2(data,'性价比')
print(data.head())

   类别   口味   环境   服务  人均消费       性价比
0  烧烤  8.0  8.6  7.9   124  0.197581
2  粤菜  6.7  7.2  6.6   141  0.145390
3  海鲜  7.2  7.2  7.3   148  0.146622
4  烧烤  7.2  7.6  7.0   143  0.152448
5  本菜  8.2  8.9  8.4    66  0.386364


In [218]:
# （4）分组——均值——标准化 

def f3(data,col):
    col_name = col + '_norm'
    data_gy = data.groupby('类别').mean()
    data_gy.sort_values(by = col_name, inplace = True, ascending=False)
    min_c= data_gy[col_name].min()
    max_c= data_gy[col_name].max()
#     data_gy_b = data_gy[(data_gy[col]-min_c)/(max_c-min_c)][['类别','服务']]
#     data_gy[col_name] = (data_gy[col] - min_c) / (max_c-min_c)
    return (data_gy[col_name] - min_c) / (max_c-min_c)
print('Yes!')

Yes!


In [219]:
data = data.groupby('类别').mean()
data['口味_norm'] = (data['口味'] - data['口味'].min() )/ (data['口味'].max()-data['口味'].min())
data['性价比_norm'] = (data['性价比'] - data['性价比'].min() )/ (data['性价比'].max()-data['性价比'].min())
data['人均消费_norm'] = (data['人均消费'] - data['人均消费'].min() )/ (data['人均消费'].max()-data['人均消费'].min())
data_final_q = data[['口味','口味_norm','性价比','性价比_norm','人均消费','人均消费_norm']]
print(data_final_q)

          口味   口味_norm       性价比  性价比_norm        人均消费  人均消费_norm
类别                                                               
亚菜  7.889937  0.814421  0.317688  0.168333  101.958071   0.358397
助餐  7.394745  0.117006  0.236529  0.082422  141.484401   0.544886
北菜  7.350125  0.054164  0.572403  0.437962   50.478750   0.115512
午茶  7.500000  0.265244  0.158667  0.000000  150.000000   0.585063
南菜  7.984874  0.948128  0.325731  0.176847   91.411765   0.308638
啡厅  7.630942  0.449659  0.601575  0.468841   46.251325   0.095566
川菜  7.489197  0.250029  0.427701  0.284787   64.149872   0.180013
州菜  7.772500  0.649026  0.441502  0.299395   71.125000   0.212923
常菜  7.311667  0.000000  0.611388  0.479230   46.258333   0.095599
快餐  7.361730  0.070508  1.103353  1.000000   25.996115   0.000000
料理  7.770436  0.646118  0.403445  0.259110   76.457008   0.238080
本菜  7.959824  0.912848  0.287742  0.136633  155.530553   0.611157
浙菜  7.503864  0.270686  0.336250  0.187982  102.991499   0.363272
海鲜  7.6931

In [220]:
# （5）开始以图做参考

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource
# 添加Tool工具、十字标线
from bokeh.models import HoverTool
from bokeh.models.annotations import BoxAnnotation

output_file('project07_h1,html')
data_final_q['size'] = data_final_q['口味_norm']*40    # 乘以40，只是为了让数字本省扩大一点
data_final_q.index.name = 'type'
data_final_q.columns = ['kw','kw_norm','xjb','xjb_norm','price','price_norm','size']
print(data_final_q)


source = ColumnDataSource(data_final_q)   #创建数据
hover = HoverTool(tooltips =[('餐饮类型','@type'),
                             ('人均消费','@price'),
                             ('性价比得分','@xjb_norm'),
                             ('口味得分','@kw_norm')])  # 把Tools加到result、kw、price上

            kw   kw_norm       xjb  xjb_norm       price  price_norm  \
type                                                                   
亚菜    7.889937  0.814421  0.317688  0.168333  101.958071    0.358397   
助餐    7.394745  0.117006  0.236529  0.082422  141.484401    0.544886   
北菜    7.350125  0.054164  0.572403  0.437962   50.478750    0.115512   
午茶    7.500000  0.265244  0.158667  0.000000  150.000000    0.585063   
南菜    7.984874  0.948128  0.325731  0.176847   91.411765    0.308638   
啡厅    7.630942  0.449659  0.601575  0.468841   46.251325    0.095566   
川菜    7.489197  0.250029  0.427701  0.284787   64.149872    0.180013   
州菜    7.772500  0.649026  0.441502  0.299395   71.125000    0.212923   
常菜    7.311667  0.000000  0.611388  0.479230   46.258333    0.095599   
快餐    7.361730  0.070508  1.103353  1.000000   25.996115    0.000000   
料理    7.770436  0.646118  0.403445  0.259110   76.457008    0.238080   
本菜    7.959824  0.912848  0.287742  0.136633  155.530553    0.61

In [221]:

# 散点图
result = figure(plot_width = 800,
                plot_height = 300,
                title = '餐饮类型得分',
                x_axis_label = '人均消费',
                y_axis_label = '性价比得分',
                tools=[hover,'box_select,reset,xwheel_zoom,pan,crosshair'])
result.circle(x= 'price' ,
              y = 'xjb_norm',
              source = source, 
              line_color = 'black',
              line_dash = [6,4],
              fill_alpha = 0.6 ,
              size = 'size')

# 柱状图1
data_type = data_final_q.index.tolist()     # 因为呈现的横坐标为文字，所以先要将index转化成list

kw = figure(plot_width = 800,
            plot_height = 300,
            title = '餐饮类型得分',
            x_range = data_type,tools=[hover,'box_select,reset,xwheel_zoom,pan,crosshair'])
kw.vbar(x = 'type',top = 'kw_norm',source = source,width = 0.8,alpha = 0.7,color = 'red')

# 柱状图2
price = figure(plot_width = 800,
               plot_height = 300,
               title = '人均消费得分',
               x_range = data_type,
               tools=[hover,'box_select,reset,xwheel_zoom,pan,crosshair'])
price.vbar(x = 'type',top = 'price_norm',source = source,width = 0.8,alpha = 0.7,color = 'green')

p = gridplot([[result],[kw],[price]])
show(p)